In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB046710,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298050,Mr,Peter,Tate,Male,...,2025-04-29 00:00:00,8,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KB049782,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298050,Mr,Peter,Tate,Male,...,2025-04-29 00:00:00,8,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
2,KK073164,Karma Resorts,Other,Past Guests,Karma Kandara,PF310816,Mr,Floris / Kanokporn,Pituktae,Male,...,2025-04-27 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,7,0,0,Email and Phone,NO
3,KB049983,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287370,Mr,John,Newey,Male,...,2025-05-02 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
4,KSM011740,Karma Resorts,Other,Past Guests,Karma St Martins,PF312957,Ms,Jean,Deacon,Female,...,2025-04-30 00:00:00,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,KK071532,Karma Resorts,Other,Past Guests,Karma Kandara,PF307149,Ms,Fadwa,Altuwaijri,Unknown,...,2025-05-02 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Email Only,NO
150,KB050314,Karma Resorts,Other,Past Guests,Karma Bavaria,PF165494,Mr,Anton,Tardif,Unknown,...,2025-05-02 00:00:00,1,COMP,STAFF,COMP,1,0,0,Email Only,NO
151,KLM005602,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,<NA>,<NA>,<NA>,Joe Finnegan,Unknown,...,2025-05-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email Only,NO
152,KSH020938,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292217,<NA>,Keith,Tucker,Unknown,...,2025-05-03 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:94: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
1,KB049782,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298050,Mr,Peter,Tate,Male,...,2025-04-29 00:00:00,8,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
10,KK073886,Karma Resorts,Other,Past Guests,Karma Kandara,PF227672,Mr.,Eric Rishi,Sami,Male,...,2025-04-30 00:00:00,2,DIR,CHAT,DIGITAL,2,0,0,Email and Phone,NO
18,KK072785,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
19,KK073111,Karma Resorts,Other,Past Guests,Karma Kandara,PF310743,Mr,Yaojia,Chen,Male,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
20,KK073087,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,KK071532,Karma Resorts,Other,Past Guests,Karma Kandara,PF307149,Ms,Fadwa,Altuwaijri,Unknown,...,2025-05-02 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Email Only,NO
150,KB050314,Karma Resorts,Other,Past Guests,Karma Bavaria,PF165494,Mr,Anton,Tardif,Unknown,...,2025-05-02 00:00:00,1,COMP,STAFF,COMP,1,0,0,Email Only,NO
151,KLM005602,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,empty,empty,empty,Joe Finnegan,Unknown,...,2025-05-02 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email Only,NO
152,KSH020938,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292217,empty,Keith,Tucker,Unknown,...,2025-05-03 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email Only,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB046710,Karma Resorts,Other,Past Guests,Karma Bavaria,PF298050,Mr,Peter,Tate,Male,...,2025-04-29 00:00:00,8,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
1,KK073164,Karma Resorts,Other,Past Guests,Karma Kandara,PF310816,Mr,Floris / Kanokporn,Pituktae,Male,...,2025-04-27 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,7,0,0,Email and Phone,NO
2,KB049983,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287370,Mr,John,Newey,Male,...,2025-05-02 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email and Phone,NO
3,KSM011740,Karma Resorts,Other,Past Guests,Karma St Martins,PF312957,Ms,Jean,Deacon,Female,...,2025-04-30 00:00:00,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
4,KK072819,Karma Resorts,Other,Past Guests,Karma Kandara,PF309946,Mrs,Helen,Cruse,Female,...,2025-04-30 00:00:00,8,HOUSE_USE,V_OWNER_NRB,COMP,4,1,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,KSM011743,Karma Resorts,Other,Past Guests,Karma St Martins,PF312950,Ms,Linda,Carter,Female,...,2025-04-30 00:00:00,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
80,KSM011744,Karma Resorts,Other,Past Guests,Karma St Martins,PF312949,Mr,Drew,Fedorka,Male,...,2025-04-30 00:00:00,2,TA,TEL,RETAIL,1,0,0,Email and Phone,NO
81,KSM011742,Karma Resorts,Other,Past Guests,Karma St Martins,PF312951,Ms,Jean,Gierlich,Female,...,2025-04-30 00:00:00,2,TA,TEL,RETAIL,2,0,0,Email and Phone,NO
82,KK073546,Karma Resorts,Other,Past Guests,Karma Kandara,PF312069,Mr.,Kunal,Jain,Male,...,2025-04-30 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
20,KB048749,Karma Resorts,Other,Past Guests,Karma Bavaria,PF224296,empty,Gerd,Schniering,Male,...,2025-05-02 00:00:00,3,COMP,V_OWNER,COMP,6,0,0,Email and Phone,NO
36,KK073566,Karma Resorts,Other,Past Guests,Karma Kandara,PF312099,Mr.,Andy,Ratsimanohatra,Male,...,2025-04-28 00:00:00,4,OTA,OTA,RETAIL,5,1,0,Email and Phone,NO
38,KB050299,Karma Resorts,Other,Past Guests,Karma Bavaria,PF304834,Mrs,Sarika,Gulati,Female,...,2025-04-28 00:00:00,1,COMP,OWNER,KC,2,0,0,Email and Phone,NO
47,KB050082,Karma Resorts,Other,Past Guests,Karma Bavaria,PF305002,empty,Peter James,Goldstone,Unknown,...,2025-05-01 00:00:00,11,STAFF_PRIV,STAFF,COMP,1,0,0,Email and Phone,NO
49,KK065374,Karma Resorts,Other,Past Guests,Karma Kandara,PF098789,Miss,Margarita,Kislitsyna,Female,...,2025-05-03 00:00:00,22,HOUSE_USE,V_OWNER,COMP,10,0,0,Email and Phone,NO
54,KB048963,Karma Resorts,Other,Past Guests,Karma Bavaria,PF276224,Ms,Eva-Maria,Boerschlein,Female,...,2025-05-01 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
78,KK072761,Karma Resorts,Other,Past Guests,Karma Kandara,PF226486,Mr,Brian,Bigos,Male,...,2025-05-01 00:00:00,6,DIR,CHAT,DIGITAL,4,2,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB050177,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312431,Frau,Dinah,Wijsenbeek,Female,...,2025-04-27 00:00:00,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
1,KB044817,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288304,Frau,Franziska,Tropcic,Female,...,2025-04-27 00:00:00,1,OTA,OTA,RETAIL,5,0,0,Email and Phone,NO
2,KB046535,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312741,Frau,Bernadette,Krummenacher,Female,...,2025-04-27 00:00:00,1,TA,TA,RETAIL,2,0,0,Email Only,NO
3,KB049997,Karma Resorts,Other,Past Guests,Karma Bavaria,PF311486,Frau,Maria,Resch,Female,...,2025-04-27 00:00:00,1,OTA,OTA,RETAIL,2,0,1,Email and Phone,NO
4,KSM015380,Karma Resorts,Other,Past Guests,Karma St Martins,PF312067,Dr,Nicola,Heneghan,Female,...,2025-04-27 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,KK073147,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
73,KK073197,Karma Resorts,Other,Past Guests,Karma Kandara,PF310743,Mr,Yaojia,Chen,Male,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
74,KK072921,Karma Resorts,Other,Past Guests,Karma Kandara,PF309166,Mr,Haiqi,Yang,Male,...,2025-05-05 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
75,KK073289,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Mr,Yaojia,Chen,Male,...,2025-05-05 00:00:00,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_10404\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB050177,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312431,Frau,Dinah,Wijsenbeek,Female,...,2025-04-27,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
1,KB044817,Karma Resorts,Other,Past Guests,Karma Bavaria,PF288304,Frau,Franziska,Tropcic,Female,...,2025-04-27,1,OTA,OTA,RETAIL,5,0,0,Email and Phone,NO
2,KB046535,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312741,Frau,Bernadette,Krummenacher,Female,...,2025-04-27,1,TA,TA,RETAIL,2,0,0,Email Only,NO
3,KB049997,Karma Resorts,Other,Past Guests,Karma Bavaria,PF311486,Frau,Maria,Resch,Female,...,2025-04-27,1,OTA,OTA,RETAIL,2,0,1,Email and Phone,NO
4,KSM015380,Karma Resorts,Other,Past Guests,Karma St Martins,PF312067,Dr,Nicola,Heneghan,Female,...,2025-04-27,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,KK073289,Karma Resorts,Other,Past Guests,Karma Kandara,PF311272,Mr,Yaojia,Chen,Male,...,2025-05-05,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
73,KK073147,Karma Resorts,Other,Past Guests,Karma Kandara,PF309894,Mr,Haiqi,Yang,Male,...,2025-05-05,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
74,KK073197,Karma Resorts,Other,Past Guests,Karma Kandara,PF310743,Mr,Yaojia,Chen,Male,...,2025-05-05,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
75,KK072921,Karma Resorts,Other,Past Guests,Karma Kandara,PF309166,Mr,Haiqi,Yang,Male,...,2025-05-05,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 154
Jumlah Data Yang Duplicate : 70
Jumlah Data Setelah Hapus Duplicate : 84
Jumlah Data Yang Termasuk Member: 7
Jumlah Data Setelah Hapus Member : 77
Jumlah Data Akhir : 77


In [19]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)